In [11]:
# Install dependencies as needed:
# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path = "sample.csv"

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "thoughtvector/customer-support-on-twitter",file_path)

  # Provide any additional arguments like
  # sql_query or pandas_kwargs. See the
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas

df.head(5)

<ipython-input-11-2608642126>:10: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df = kagglehub.load_dataset(


,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,119237,105834,True,Wed Oct 11 06:55:44 +0000 2017,@AppleSupport causing the reply to be disregar...,119236,NaN
1,119238,ChaseSupport,False,Wed Oct 11 13:25:49 +0000 2017,@105835 Your business means a lot to us. Pleas...,NaN,119239.0
2,119239,105835,True,Wed Oct 11 13:00:09 +0000 2017,@76328 I really hope you all change but I'm su...,119238,NaN
3,119240,VirginTrains,False,Tue Oct 10 15:16:08 +0000 2017,@105836 LiveChat is online at the moment - htt...,119241,119242.0
4,119241,105836,True,Tue Oct 10 15:17:21 +0000 2017,@VirginTrains see attached error message. I've...,119243,119240.0


In [12]:
df.columns


Index(['tweet_id', 'author_id', 'inbound', 'created_at', 'text',
       'response_tweet_id', 'in_response_to_tweet_id'],
      dtype='object')

In [13]:
# Filter customer and support messages
customers = df[df['inbound'] == True]
support = df[df['inbound'] == False]

In [14]:
import pandas as pd


In [15]:
# Ensure both keys are strings before merging
customers['response_tweet_id'] = customers['response_tweet_id'].astype(str)
support['tweet_id'] = support['tweet_id'].astype(str)

# Now merge safely
chat_pairs = pd.merge(
    customers,
    support,
    left_on='response_tweet_id',
    right_on='tweet_id',
    suffixes=('_customer', '_support')
)

# Keep only needed columns
chat_pairs = chat_pairs[['text_customer', 'text_support']]
chat_pairs.columns = ['customer_query', 'support_reply']

# Show some sample pairs
chat_pairs.head()


<ipython-input-15-1033481335>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  customers['response_tweet_id'] = customers['response_tweet_id'].astype(str)
<ipython-input-15-1033481335>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  support['tweet_id'] = support['tweet_id'].astype(str)


,customer_query,support_reply
0,@76328 I really hope you all change but I'm su...,@105835 Your business means a lot to us. Pleas...
1,@VirginTrains see attached error message. I've...,"@105836 Have you tried from another device, Mi..."
2,"@VirginTrains yep, I've tried laptop too sever...","@105836 It's working OK from here, Miriam. Doe..."
3,@VirginTrains I still haven't heard &amp; the ...,@105836 LiveChat is online at the moment - htt...
4,"@105838 @AppleSupport Me too am suffering , ho...",@105837 We can help. Which version of iOS are ...


In [16]:
!pip install -q transformers datasets



In [17]:
from datasets import Dataset

# Truncate long examples to keep it small (optional)
chat_pairs = chat_pairs.dropna().drop_duplicates()
chat_pairs = chat_pairs[chat_pairs['customer_query'].str.len() < 300]
chat_pairs = chat_pairs[chat_pairs['support_reply'].str.len() < 300]

# Hugging Face dataset format
dataset = Dataset.from_pandas(chat_pairs)


In [18]:
from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained("t5-small")

def preprocess(example):
    input_text = "question: " + example["customer_query"]
    target_text = example["support_reply"]

    model_input = tokenizer(
        input_text,
        padding="max_length",
        max_length=128,
        truncation=True
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            target_text,
            padding="max_length",
            max_length=128,
            truncation=True
        )

    model_input["labels"] = labels["input_ids"]
    return model_input

tokenized_dataset = dataset.map(preprocess)


Map:   0%|          | 0/37 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3959: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [19]:
from datasets import Dataset
from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained("t5-small")

def preprocess(example):
    input_text = "question: " + example["customer_query"]
    target_text = example["support_reply"]

    model_input = tokenizer(
        input_text,
        padding="max_length",
        max_length=128,
        truncation=True
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            target_text,
            padding="max_length",
            max_length=128,
            truncation=True
        )

    model_input["labels"] = labels["input_ids"]
    return model_input

tokenized_dataset = dataset.map(preprocess)

Map:   0%|          | 0/37 [00:00<?, ? examples/s]

In [31]:
from transformers import T5ForConditionalGeneration

model = T5ForConditionalGeneration.from_pretrained("t5-small")


In [35]:
from transformers import TrainingArguments, Trainer

from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./t5-chatbot",
    per_device_train_batch_size=4,
    num_train_epochs=10,
    logging_steps=10,
    save_steps=100,)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
)

trainer.train()

Step,Training Loss
10,4.070900
20,2.796700
30,2.375400
40,2.099300
50,1.970100
60,1.900400
70,1.777400
80,1.857400
90,1.774000
100,1.737400


TrainOutput(global_step=100, training_loss=2.2358959770202635, metrics={'train_runtime': 567.0464, 'train_samples_per_second': 0.653, 'train_steps_per_second': 0.176, 'total_flos': 12519116636160.0, 'train_loss': 2.2358959770202635, 'epoch': 10.0})

In [40]:
input_text = "question: Where is my package?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids
outputs = model.generate(input_ids, max_length=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
